# Model Management in Dataloop

In [59]:
#importing Dataloop's Python Package, Torch and OS
import dtlpy as dl
import torch
import os
import adapter_script

In [60]:
#Logging in to Dataloop (checks if token expired ~24h expiration time for token)
if dl.token_expired():
   dl.login()
#you can also use the simple login: 
#dl.login()

## Important Pre-requisite!

Before you can use everything below, you need to create a `.py` file called `adapter_script.py` inside of your working directory (so it can be accessible), containing the following code:

-----------------------------------------------------------------------------------------------
```python

import dtlpy as dl
import torch
import os
@dl.Package.decorators.module(name='model-adapter',
                              description='Model Adapter for my model',
                              init_inputs={'model_entity': dl.Model})
class SimpleModelAdapter(dl.BaseModelAdapter):
    def load(self, local_path, **kwargs):
        print('loading a model')
        self.model = torch.load(os.path.join(local_path, 'model.pth')) #here you place your model file's name and extension
    def predict(self, batch, **kwargs):
        print('predicting batch of size: {}'.format(len(batch)))
        preds = self.model(batch)
        batch_annotations = list()
        for i_img, predicted_class in enumerate(preds):  # annotations per image
            image_annotations = dl.AnnotationCollection()
            # in this example, we will assume preds is a label for a classification model
            image_annotations.add(annotation_definition=dl.Classification(label=predicted_class),
                                  model_info={'name': self.model_name})
            batch_annotations.append(image_annotations)
        return batch_annotations

```
------------------------------------------------------------------------------------------------------
**Note:** You need to have PyTorch installed. [Find out how to install it here](https://pytorch.org/get-started/locally/).

The simplest way to create this file is to create a `.txt` file, open it, copy paste this code inside of it, and then change its file extension from `.tx`t to `.py`

In [61]:
# Create the package

import dtlpy as dl
from adapter_script import SimpleModelAdapter
project = dl.projects.get(project_name='CreatureHunt')
dataset = project.datasets.get(dataset_name='Creatures')
# codebase = project.codebases.pack(directory='<path to local dir>')
# codebase: dl.GitCodebase = dl.GitCodebase(git_url='github.com/mygit', git_tag='v25.6.93')
metadata = dl.Package.get_ml_metadata(cls=SimpleModelAdapter,
                                      default_configuration={},
                                      output_type=dl.AnnotationType.CLASSIFICATION
                                      )
module = dl.PackageModule.from_entry_point(entry_point='adapter_script.py')

In [62]:
# Push the Package
package = project.packages.push(package_name='package11',
                                src_path=os.getcwd(),
                                package_type='ml',
                                # codebase=codebase,
                                modules=[module],
                                is_global=False,
                                service_config={
                                    'runtime': dl.KubernetesRuntime(pod_type=dl.INSTANCE_CATALOG_GPU_K80_S,
                                                                    autoscaler=dl.KubernetesRabbitmqAutoscaler(
                                                                        min_replicas=0,
                                                                        max_replicas=1),
                                                                    concurrency=1).to_json()},
                                metadata=metadata)


[2023-05-02 21:55:05][WAR][dtlpy:v1.76.15][repositories.packages:1613] missing input name: "cleanup" in function definition: "evaluate_model"
[2023-05-02 21:55:05][WAR][dtlpy:v1.76.15][repositories.packages:1613] missing input name: "with_upload" in function definition: "predict_dataset"
[2023-05-02 21:55:05][WAR][dtlpy:v1.76.15][repositories.packages:1613] missing input name: "cleanup" in function definition: "predict_dataset"
[2023-05-02 21:55:05][WAR][dtlpy:v1.76.15][repositories.packages:1613] missing input name: "batch_size" in function definition: "predict_dataset"
[2023-05-02 21:55:05][WAR][dtlpy:v1.76.15][repositories.packages:1613] missing input name: "output_shape" in function definition: "predict_dataset"
[2023-05-02 21:55:05][WAR][dtlpy:v1.76.15][repositories.packages:1613] missing input name: "upload_annotations" in function definition: "predict_items"
[2023-05-02 21:55:05][WAR][dtlpy:v1.76.15][repositories.packages:1613] missing input name: "conf_threshold" in function de

Upload Items: 100%|██████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.50s/it]


## Upload artifacts and create the model

Now you can create a model and upload pretrained model weights with an Artifact Item. Here, the Artfiact item is where the saved model weights are. You can upload any weights file here and name it according to the 'weights_filename' in the configuration.

In [63]:

artifact = dl.LocalArtifact(local_path='/model.h5')
model = package.models.create(model_name='magic-creatures_2',
                              description='magical creatures model',
                              tags=['pretrained', 'tutorial','onboarding'],
                              dataset_id='63e6283b4a03c631b54725ec',
                              configuration={'weights_filename': 'model.h5'
                                             },
                              project_id=package.project.id,
                              model_artifacts=[artifact],
                              labels=['Bear With Wings', 'Blue Panther with Wings', 'Blue Tiger With Wings','Elephant ','Fox','Giraffe','Kitty ','Koala','Lion','Unicorn']
                              )


[2023-05-02 21:55:18][WAR][dtlpy:v1.76.15][repositories.models:352] Model is using an unlocked dataset 'Creatures'. Make it readonly for training reproducibility


## Testing the model

To deploy a model, its status must be set to trained so you can deploy a model by updating the status to trained and then deploy it.


In [64]:
model.status = 'trained'
model.update()
model.deploy()

True

Now, we can actually use it to predict an Item. But first, we need to find out the model's ID and pull out an Item's ID to run the prediction on it. To find the Model's ID, you can simply print it, since we already have it as part of a variable. To find the Item id for an Item we want to run the prediction on, it's best to just open the Dataset in web and look for the Item by going on the left side of the screen (after selecting your project), clicking Data Management, Datasets, and then doubleclicking your dataset - now you should see all items in that Dataset. After clicking an Item you should see the item ID.

In [65]:
print(model.id)

64515c95a50ccd321591f854


In [11]:
dataset.open_in_web()

Now that we have the Model ID and Item ID ready, we can execute a prediction on an item

In [68]:

model = dl.models.get(model_id='64515c95a50ccd321591f854')
item = dl.items.get(item_id='63eb694cbb4d8448e0878711')
model.deploy()
execution = model.predict(item_ids=[item.id])
# after a few seconds, update your execution from the cloud
execution = dl.executions.get(execution_id=execution.id)
# print the most recent status
print(execution.status[-1]['status'])


created


In [69]:
print(model.predict(item_ids=[item.id]))

Execution(id='64515d121931d4593a9291d5', creator='myfuncont@gmail.com', created_at='2023-05-02T18:57:22.626Z', input={'items': ['63eb694cbb4d8448e0878711']}, latest_status={'timestamp': '2023-05-02T18:57:22.626Z', 'status': 'created', 'percentComplete': None, 'error': None, 'output': None}, function_name='predict_items', duration=None, attempts=0, max_attempts=3, to_terminate=False, trigger_id=None, service_id='64515c9f9d6716027513f24b', project_id='4c74c1b5-e9cb-4294-b9d5-cbfa13eda242', service_version='1.0.0', package_id='64515c8b9d6716937313f24a', package_name='package11')


In [23]:
model.deploy()

True

In [25]:
model = dl.models.get(model_id='6450f6e1aad96a0deec8f13c')


In [26]:
execution = model.predict(item_ids=[item.id])

In [27]:
print(execution)

Execution(id='6450f71482d4277ba2f2a2ba', creator='myfuncont@gmail.com', created_at='2023-05-02T11:42:12.140Z', input={'items': ['63eb694cbb4d8448e0878711']}, latest_status={'timestamp': '2023-05-02T11:42:12.140Z', 'status': 'created', 'percentComplete': None, 'error': None, 'output': None}, function_name='predict_items', duration=None, attempts=0, max_attempts=3, to_terminate=False, trigger_id=None, service_id='6450f6ea014e51186820e1ad', project_id='4c74c1b5-e9cb-4294-b9d5-cbfa13eda242', service_version='1.0.0', package_id='644f8e764f2d0c2cdf52b438', package_name='package1')


In [28]:
model.dataset

Dataset(id='63e6283b4a03c631b54725ec', url='https://gate.dataloop.ai/api/v1/datasets/63e6283b4a03c631b54725ec', name='Creatures', creator='myfuncont@gmail.com', items_count=1132, expiration_options=None, index_driver='v1', created_at='2023-02-10T11:19:23.239Z')

In [47]:

package = dl.packages.get(package_id=model.package_id)
adapter = package.build(module_name='model-adapter')
adapter.load_from_model(model_entity=model)


Download Items: 100%|████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  7.29it/s]
loading a model


UnpicklingError: invalid load key, 'H'.